In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [ ]:
from keras.preprocessing import image        

In [ ]:
from keras.models import load_model
import h5py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

filename = '/content/drive/MyDrive/Broner/InceptionResnet/Ml/finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
m = load_model('/content/drive/MyDrive/Broner/model.h5')
m1 = load_model('/content/drive/MyDrive/Broner/InceptionResnet/0.hdf5')
m2 = load_model('/content/drive/MyDrive/Broner/InceptionResnet/1.hdf5')
m3 = load_model('/content/drive/MyDrive/Broner/InceptionResnet/2.hdf5')
m4 = load_model('/content/drive/MyDrive/Broner/InceptionResnet/3.hdf5')
m5 = load_model('/content/drive/MyDrive/Broner/InceptionResnet/4.hdf5')

In [ ]:
def check_value(var):
  
  if var>0.5:
    return 1
  return 0
def new_answer(img):
  arr = np.empty(5, dtype=int)
  # img = image.load_img(path,target_size=(320,320))
  img_tensor = image.img_to_array(img)
  img_tensor = np.expand_dims(img_tensor,axis = 0)
  img_tensor /= 255
  img_tensor[:,:,0] = (img_tensor[:,:,0]-0.485)/0.229
  img_tensor[:,:,1] = (img_tensor[:,:,1]-0.456)/0.224
  img_tensor[:,:,2] = (img_tensor[:,:,2]-0.406)/0.225
  ans1 = m1.predict(img_tensor)
  arr[0] = check_value(ans1)
  ans2 = m2.predict(img_tensor) 
  arr[1] = check_value(ans2)
  ans3 = m3.predict(img_tensor)
  arr[2] = check_value(ans3)
  ans4 = m4.predict(img_tensor)
  arr[3] = check_value(ans4)
  ans5 = m5.predict(img_tensor)
  arr[4] = check_value(ans5)
  arr = arr.reshape(1,-1)
  ans = m.predict(img_tensor)
  a = np.argmax(ans)
  if a == 0:
    bone = 'Shoulder'
  elif a == 1:
    bone = 'Humerus' 
  else:
    bone = 'Forearm'
  # print(loaded_model.predict(arr))
  return loaded_model.predict(arr),bone

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok
import cv2,base64,re
import werkzeug
import keras.models
import numpy
import scipy.misc
app = Flask(__name__,template_folder='/content/drive/MyDrive/templates',static_folder='/content/drive/MyDrive/static')
run_with_ngrok(app)   
# app.debug = True
@app.route("/")
def home():
    return render_template('index.html')


@app.route("/image",methods=['POST','GET'])
def answers():
  if request.method == 'POST':
    imagefile = request.files['img']
    filename = 'temp.jpg'
    print("\nReceived image File name : " + imagefile.filename)
    imagefile.save(filename)
    
    img = cv2.imread(filename)
    resized = cv2.resize(img, (320,320))

    # img = request.files['img']
    # r = base64.b64decode(img.read())
    # image = np.asarray(bytearray(r), dtype="uint8")
    # image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # resized = cv2.resize(image, (320,320))
    # print(resized)
    result,bone = new_answer(resized)
    if result[0] == 0:
      rs = 'negative'
    else:
      rs = 'positive'
    return render_template('index.html',results = rs,bone = bone)
  else:
    return render_template('index.html')
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5afb255e8be7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Apr/2021 15:14:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:14:23] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:14:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2021 15:14:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:14:25] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:14:57] "GET / HTTP/1.1" 200 -



Received image File name : image1.png


127.0.0.1 - - [21/Apr/2021 15:15:16] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:15:18] "GET /static/style.css HTTP/1.1" 200 -


127.0.0.1 - - [21/Apr/2021 15:15:40] "POST /image HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:15:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:15:53] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 15:15:53] "GET /favicon.ico HTTP/1.1" 404 -



Received image File name : 9.jpg


127.0.0.1 - - [21/Apr/2021 15:16:08] "POST /image HTTP/1.1" 200 -
